In [ ]:
import flair

In [6]:
import pandas as pd
from tqdm import tqdm
from difflib import SequenceMatcher
import re
import pickle

In [11]:
import flair
from flair.data import Corpus
from flair.datasets import ColumnCorpus
columns = {0: 'text', 1: 'ner'}
corpus: Corpus = ColumnCorpus('data/', columns,
                              train_file='flairfirspart.txt',
                              test_file='flairtest2.txt',
                              dev_file='flairval2.txt')

2022-01-04 16:52:16,623 Reading data from data
2022-01-04 16:52:16,624 Train: data\flairfirspart.txt
2022-01-04 16:52:16,625 Dev: data\flairval2.txt
2022-01-04 16:52:16,627 Test: data\flairtest2.txt


In [12]:
# tag to predict
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(tag_type)

2022-01-04 16:52:18,044 Computing label dictionary. Progress:


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3173/3173 [00:00<00:00, 11332.15it/s]

2022-01-04 16:52:18,328 Corpus contains the labels: ner (#75237)
2022-01-04 16:52:18,329 Created (for label 'ner') Dictionary with 7 tags: <unk>, O, -, U-PERSON, B-PERSON, L-PERSON, I-PERSON


In [13]:
from flair.embeddings import WordEmbeddings, StackedEmbeddings, TokenEmbeddings
from typing import List
embedding_types : List[TokenEmbeddings] = [
        WordEmbeddings('glove'),
        ## other embeddings
        ]
embeddings : StackedEmbeddings = StackedEmbeddings(
                                 embeddings=embedding_types)

In [14]:
from flair.models import SequenceTagger
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)

SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=9, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)


In [15]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
    
trainer.train('data/flairfirstpartmodel',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2022-01-04 16:52:33,039 ----------------------------------------------------------------------------------------------------
2022-01-04 16:52:33,041 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=9, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2022-01-04 16:52:33,042 ----------------------------------------------------------------------------------------------------
2022-01-04 16:52:33,044 Corpus: "Corpus: 3173 train + 546 dev + 438 test sentences"
2022-01-04 16:52:33,045 ----------------------------------------------------------------------------------------------------
2022-01-04 16:52:33

2022-01-04 16:58:19,224 epoch 5 - iter 30/100 - loss 0.02583411 - samples/sec: 43.91 - lr: 0.100000
2022-01-04 16:58:26,520 epoch 5 - iter 40/100 - loss 0.02617781 - samples/sec: 43.88 - lr: 0.100000
2022-01-04 16:58:35,538 epoch 5 - iter 50/100 - loss 0.02410762 - samples/sec: 35.49 - lr: 0.100000
2022-01-04 16:58:41,995 epoch 5 - iter 60/100 - loss 0.02619052 - samples/sec: 49.57 - lr: 0.100000
2022-01-04 16:58:48,731 epoch 5 - iter 70/100 - loss 0.02647626 - samples/sec: 47.51 - lr: 0.100000
2022-01-04 16:58:56,501 epoch 5 - iter 80/100 - loss 0.02686866 - samples/sec: 41.20 - lr: 0.100000
2022-01-04 16:59:04,077 epoch 5 - iter 90/100 - loss 0.02771354 - samples/sec: 42.26 - lr: 0.100000
2022-01-04 16:59:10,975 epoch 5 - iter 100/100 - loss 0.02721265 - samples/sec: 46.41 - lr: 0.100000
2022-01-04 16:59:10,977 ----------------------------------------------------------------------------------------------------
2022-01-04 16:59:10,978 EPOCH 5 done: loss 0.0272 - lr 0.1000000
2022-01-0

2022-01-04 17:05:46,325 ----------------------------------------------------------------------------------------------------
2022-01-04 17:05:46,326 EPOCH 10 done: loss 0.0224 - lr 0.1000000
2022-01-04 17:05:49,571 DEV : loss 0.0187847763299942 - f1-score (micro avg)  0.6763
2022-01-04 17:05:49,611 BAD EPOCHS (no improvement): 1
2022-01-04 17:05:49,613 ----------------------------------------------------------------------------------------------------
2022-01-04 17:05:57,882 epoch 11 - iter 10/100 - loss 0.02683501 - samples/sec: 38.72 - lr: 0.100000
2022-01-04 17:06:05,357 epoch 11 - iter 20/100 - loss 0.02592722 - samples/sec: 42.82 - lr: 0.100000
2022-01-04 17:06:12,606 epoch 11 - iter 30/100 - loss 0.02465658 - samples/sec: 44.15 - lr: 0.100000
2022-01-04 17:06:20,216 epoch 11 - iter 40/100 - loss 0.02657129 - samples/sec: 42.06 - lr: 0.100000
2022-01-04 17:06:27,466 epoch 11 - iter 50/100 - loss 0.02622253 - samples/sec: 44.14 - lr: 0.100000
2022-01-04 17:06:34,393 epoch 11 - iter

2022-01-04 17:12:50,332 epoch 16 - iter 40/100 - loss 0.01958036 - samples/sec: 43.87 - lr: 0.050000
2022-01-04 17:12:56,576 epoch 16 - iter 50/100 - loss 0.01988763 - samples/sec: 51.25 - lr: 0.050000
2022-01-04 17:13:02,810 epoch 16 - iter 60/100 - loss 0.01854202 - samples/sec: 51.36 - lr: 0.050000
2022-01-04 17:13:09,802 epoch 16 - iter 70/100 - loss 0.01888305 - samples/sec: 45.79 - lr: 0.050000
2022-01-04 17:13:17,002 epoch 16 - iter 80/100 - loss 0.01828588 - samples/sec: 44.46 - lr: 0.050000
2022-01-04 17:13:22,598 epoch 16 - iter 90/100 - loss 0.01793670 - samples/sec: 57.20 - lr: 0.050000
2022-01-04 17:13:29,800 epoch 16 - iter 100/100 - loss 0.01771855 - samples/sec: 44.46 - lr: 0.050000
2022-01-04 17:13:29,802 ----------------------------------------------------------------------------------------------------
2022-01-04 17:13:29,802 EPOCH 16 done: loss 0.0177 - lr 0.0500000
2022-01-04 17:13:31,942 DEV : loss 0.009220510721206665 - f1-score (micro avg)  0.9118
2022-01-04 17:

2022-01-04 17:19:26,078 EPOCH 21 done: loss 0.0168 - lr 0.0250000
2022-01-04 17:19:28,266 DEV : loss 0.008226054720580578 - f1-score (micro avg)  0.9287
2022-01-04 17:19:28,308 BAD EPOCHS (no improvement): 3
2022-01-04 17:19:28,310 ----------------------------------------------------------------------------------------------------
2022-01-04 17:19:35,593 epoch 22 - iter 10/100 - loss 0.02480870 - samples/sec: 43.95 - lr: 0.025000
2022-01-04 17:19:43,005 epoch 22 - iter 20/100 - loss 0.02053399 - samples/sec: 43.18 - lr: 0.025000
2022-01-04 17:19:50,419 epoch 22 - iter 30/100 - loss 0.01961543 - samples/sec: 43.18 - lr: 0.025000
2022-01-04 17:19:56,865 epoch 22 - iter 40/100 - loss 0.01736762 - samples/sec: 49.67 - lr: 0.025000
2022-01-04 17:20:03,341 epoch 22 - iter 50/100 - loss 0.01646511 - samples/sec: 49.43 - lr: 0.025000
2022-01-04 17:20:09,908 epoch 22 - iter 60/100 - loss 0.01752673 - samples/sec: 48.75 - lr: 0.025000
2022-01-04 17:20:17,370 epoch 22 - iter 70/100 - loss 0.01801

...

2022-01-04 17:47:11,776 epoch 44 - iter 20/100 - loss 0.01399284 - samples/sec: 45.16 - lr: 0.003125
2022-01-04 17:47:19,895 epoch 44 - iter 30/100 - loss 0.01223306 - samples/sec: 39.42 - lr: 0.003125
2022-01-04 17:47:27,969 epoch 44 - iter 40/100 - loss 0.01118540 - samples/sec: 39.65 - lr: 0.003125
2022-01-04 17:47:35,309 epoch 44 - iter 50/100 - loss 0.01228878 - samples/sec: 43.61 - lr: 0.003125
2022-01-04 17:47:42,126 epoch 44 - iter 60/100 - loss 0.01374313 - samples/sec: 46.95 - lr: 0.003125
2022-01-04 17:47:48,792 epoch 44 - iter 70/100 - loss 0.01337121 - samples/sec: 48.01 - lr: 0.003125
2022-01-04 17:47:56,307 epoch 44 - iter 80/100 - loss 0.01475504 - samples/sec: 42.59 - lr: 0.003125
2022-01-04 17:48:03,949 epoch 44 - iter 90/100 - loss 0.01512581 - samples/sec: 41.89 - lr: 0.003125
2022-01-04 17:48:10,195 epoch 44 - iter 100/100 - loss 0.01463624 - samples/sec: 51.25 - lr: 0.003125
2022-01-04 17:48:10,197 ------------------------------------------------------------------

2022-01-04 17:54:12,902 epoch 49 - iter 100/100 - loss 0.01461344 - samples/sec: 50.68 - lr: 0.001563
2022-01-04 17:54:12,903 ----------------------------------------------------------------------------------------------------
2022-01-04 17:54:12,904 EPOCH 49 done: loss 0.0146 - lr 0.0015625
2022-01-04 17:54:14,940 DEV : loss 0.007384386379271746 - f1-score (micro avg)  0.922
Epoch    49: reducing learning rate of group 0 to 7.8125e-04.
2022-01-04 17:54:14,970 BAD EPOCHS (no improvement): 4
2022-01-04 17:54:14,972 ----------------------------------------------------------------------------------------------------
2022-01-04 17:54:21,508 epoch 50 - iter 10/100 - loss 0.01800754 - samples/sec: 48.98 - lr: 0.000781
2022-01-04 17:54:28,807 epoch 50 - iter 20/100 - loss 0.01817757 - samples/sec: 43.86 - lr: 0.000781
2022-01-04 17:54:36,245 epoch 50 - iter 30/100 - loss 0.01666098 - samples/sec: 43.04 - lr: 0.000781
2022-01-04 17:54:43,820 epoch 50 - iter 40/100 - loss 0.01631031 - samples/s

2022-01-04 18:00:36,060 epoch 55 - iter 30/100 - loss 0.01322236 - samples/sec: 45.13 - lr: 0.000391
2022-01-04 18:00:43,858 epoch 55 - iter 40/100 - loss 0.01305106 - samples/sec: 41.06 - lr: 0.000391
2022-01-04 18:00:50,567 epoch 55 - iter 50/100 - loss 0.01385143 - samples/sec: 47.71 - lr: 0.000391
2022-01-04 18:00:57,432 epoch 55 - iter 60/100 - loss 0.01440648 - samples/sec: 46.63 - lr: 0.000391
2022-01-04 18:01:03,945 epoch 55 - iter 70/100 - loss 0.01538829 - samples/sec: 49.16 - lr: 0.000391
2022-01-04 18:01:11,249 epoch 55 - iter 80/100 - loss 0.01545734 - samples/sec: 43.82 - lr: 0.000391
2022-01-04 18:01:18,010 epoch 55 - iter 90/100 - loss 0.01524110 - samples/sec: 47.35 - lr: 0.000391
2022-01-04 18:01:24,140 epoch 55 - iter 100/100 - loss 0.01452865 - samples/sec: 52.22 - lr: 0.000391
2022-01-04 18:01:24,142 ----------------------------------------------------------------------------------------------------
2022-01-04 18:01:24,143 EPOCH 55 done: loss 0.0145 - lr 0.0003906


2022-01-04 18:07:23,287 ----------------------------------------------------------------------------------------------------
2022-01-04 18:07:23,288 EPOCH 60 done: loss 0.0156 - lr 0.0001953
2022-01-04 18:07:25,492 DEV : loss 0.007241406477987766 - f1-score (micro avg)  0.922
2022-01-04 18:07:25,521 BAD EPOCHS (no improvement): 3
2022-01-04 18:07:25,523 ----------------------------------------------------------------------------------------------------
2022-01-04 18:07:32,247 epoch 61 - iter 10/100 - loss 0.00803018 - samples/sec: 47.61 - lr: 0.000195
2022-01-04 18:07:39,128 epoch 61 - iter 20/100 - loss 0.01498504 - samples/sec: 46.53 - lr: 0.000195
2022-01-04 18:07:46,972 epoch 61 - iter 30/100 - loss 0.01395941 - samples/sec: 40.81 - lr: 0.000195
2022-01-04 18:07:53,005 epoch 61 - iter 40/100 - loss 0.01563273 - samples/sec: 53.06 - lr: 0.000195
2022-01-04 18:07:59,289 epoch 61 - iter 50/100 - loss 0.01519114 - samples/sec: 50.96 - lr: 0.000195
2022-01-04 18:08:06,254 epoch 61 - ite

{'test_score': 0.7382550335570471,
 'dev_score_history': [0.48125000000000007,
  0.7474226804123711,
  0.608955223880597,
  0.8,
  0.8,
  0.8605769230769231,
  0.8819277108433734,
  0.8535353535353536,
  0.9148418491484185,
  0.6762589928057553,
  0.9112709832134293,
  0.8341708542713568,
  0.8833746898263027,
  0.9297820823244553,
  0.9144254278728606,
  0.911764705882353,
  0.9170731707317074,
  0.928395061728395,
  0.9257425742574258,
  0.9203980099502488,
  0.9287469287469287,
  0.9290953545232273,
  0.928395061728395,
  0.9310344827586207,
  0.9219512195121952,
  0.9264705882352942,
  0.9219512195121952,
  0.9290953545232273,
  0.9290953545232273,
  0.9219512195121952,
  0.9290953545232273,
  0.9313725490196079,
  0.9313725490196079,
  0.9313725490196079,
  0.9313725490196079,
  0.9313725490196079,
  0.9313725490196079,
  0.9245742092457421,
  0.9219512195121952,
  0.9219512195121952,
  0.9219512195121952,
  0.9219512195121952,
  0.9219512195121952,
  0.9219512195121952,
  0.92195